In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics  import silhouette_score
from scipy.spatial.distance import cdist
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [4]:
basics_df = pd.read_csv("https://datasets.imdbws.com/title.basics.tsv.gz", sep="\t", low_memory=False)

In [5]:
basics_df =  basics_df.drop('endYear',axis=1)

In [6]:
movies = basics_df.loc[basics_df['titleType']=='movie']

In [7]:
movies = movies.drop('titleType',axis=1)

In [8]:
movies_ok = movies.loc[movies['isAdult']=='0']
movies_ok=movies_ok.drop('isAdult',axis=1)

In [9]:
movies_ok_nat = movies_ok.replace('\\N', pd.NaT)
movies_clean = movies_ok_nat.dropna()

In [10]:
movies_clean = movies_clean.astype({'runtimeMinutes': int})
movies_clean = movies_clean.astype({'startYear': int})

In [11]:
movies_clean = movies_clean[(movies_clean['runtimeMinutes'] >= 58) & (movies_clean['runtimeMinutes'] <= 270)]

In [12]:
movies_clean = movies_clean[(movies_clean['startYear'] >= 1918) & (movies_clean['startYear'] <= 2021)]

In [13]:
ratings_df = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", sep="\t")

In [14]:
movies = pd.merge(movies_clean,ratings_df,how='inner')

In [15]:
movies_7 = movies.loc[movies['averageRating']>=7.0]

In [16]:
movies_7_rating = movies_7[movies_7['numVotes'] >= 1000]


In [17]:
movies_7_rating

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
11,tt0008879,The Outlaw and His Wife,Berg-Ejvind och hans hustru,1918,136,Drama,7.2,1784
82,tt0009893,The Oyster Princess,Die Austernprinzessin,1919,60,Comedy,7.2,1817
88,tt0009937,Blind Husbands,Blind Husbands,1919,99,"Drama,Romance",7.0,1470
91,tt0009968,Broken Blossoms,Broken Blossoms or The Yellow Man and the Girl,1919,90,"Drama,Romance",7.3,10029
124,tt0010247,Sir Arne's Treasure,Herr Arnes pengar,1919,122,"Drama,History",7.2,1357
...,...,...,...,...,...,...,...,...
223064,tt9882084,Chasing Happiness,Chasing Happiness,2019,96,"Biography,Documentary,Music",7.8,2138
223072,tt9886872,Munthiri Monchan,Munthiri Monchan,2019,130,"Comedy,Romance",7.8,1345
223136,tt9900782,Kaithi,Kaithi,2019,145,"Action,Crime,Thriller",8.5,19014
223137,tt9902160,Herself,Herself,2020,97,Drama,7.0,2854


In [18]:
movie_genre = movies_7_rating[['genres']]

In [19]:
movie_genre[['genre1', 'genre2','genre3']] = movie_genre['genres'].str.split(',', n=2, expand=True)

C:\Users\josse\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [20]:
movie_genre.drop(columns='genres',axis=1,inplace=True)

C:\Users\josse\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [21]:
movie_genre = pd.concat([movie_genre , movie_genre['genre1'].str.get_dummies(), movie_genre['genre2'].str.get_dummies(), movie_genre['genre3'].str.get_dummies()], 
          axis = 1)

In [22]:
movie_genre.drop(['genre1','genre2','genre3'],axis=1,inplace=True)

In [23]:
movie_for_ml = pd.concat([movies_7_rating, movie_genre.groupby(lambda x:x, axis=1).sum()], axis = 1)

In [24]:
movie_for_ml.drop(columns=['runtimeMinutes','genres','averageRating','numVotes'],axis =1,inplace=True)

In [25]:
movie_for_ml.drop(columns='primaryTitle',axis=1,inplace=True)

In [26]:
movie_for_ml

,tconst,originalTitle,startYear,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,...,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
11,tt0008879,Berg-Ejvind och hans hustru,1918,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
82,tt0009893,Die Austernprinzessin,1919,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
88,tt0009937,Blind Husbands,1919,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
91,tt0009968,Broken Blossoms or The Yellow Man and the Girl,1919,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
124,tt0010247,Herr Arnes pengar,1919,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223064,tt9882084,Chasing Happiness,2019,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
223072,tt9886872,Munthiri Monchan,2019,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
223136,tt9900782,Kaithi,2019,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
223137,tt9902160,Herself,2020,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
movie_for_ml = movie_for_ml.reset_index(drop=True)

In [28]:
cols = movie_for_ml.columns[2:]
X = movie_for_ml[cols]

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

movie_scaled = pd.DataFrame()

movie_scaled['tconst']=movie_for_ml['tconst']
movie_scaled['originalTitle']=movie_for_ml['originalTitle']

scaled_df = pd.DataFrame(X_scaled,columns=cols)

movie_scaled_concat = pd.concat([movie_scaled,scaled_df],axis=1)

In [29]:
cols = movie_scaled_concat.columns[2:]
X = movie_scaled_concat[cols]

modelKM = KMeans(n_clusters=35)
modelKM.fit(X)

movie_scaled_concat['cluster']=modelKM.labels_

In [30]:
movie_scaled_concat

,tconst,originalTitle,startYear,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,...,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western,cluster
0,tt0008879,Berg-Ejvind och hans hustru,-2.974130,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,5
1,tt0009893,Die Austernprinzessin,-2.934086,-0.391523,-0.325588,-0.215457,-0.314509,1.587382,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,24
2,tt0009937,Blind Husbands,-2.934086,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,2.157438,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,4
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,-2.934086,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,2.157438,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,4
4,tt0010247,Herr Arnes pengar,-2.934086,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11649,tt9882084,Chasing Happiness,1.070327,-0.391523,-0.325588,-0.215457,3.179564,-0.629968,-0.436904,2.801589,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,27
11650,tt9886872,Munthiri Monchan,1.070327,-0.391523,-0.325588,-0.215457,-0.314509,1.587382,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,2.157438,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,29
11651,tt9900782,Kaithi,1.070327,2.554127,-0.325588,-0.215457,-0.314509,-0.629968,2.288833,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,3.138634,-0.214802,-0.11648,2
11652,tt9902160,Herself,1.110371,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,5


In [31]:
movie_scaled_concat.to_csv('../data/movies_for_ml.csv.zip', index=False, compression='zip')